In [8]:
import pandas as pd
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report


# Cargar datos
df = pd.read_csv("../CSV/UnidoPorAñoCSV/vif_all_years.csv")

vars_ = [
    'VIC_EDAD', 'VIC_ESCOLARIDAD', 'VIC_OCUP', 'VIC_DEDICA', 'VIC_TRABAJA',
    'AGR_EDAD', 'AGR_ESCOLARIDAD', 'AGR_OCUP', 'AGR_DEDICA', 'AGR_TRABAJA',
    'VIC_REL_AGR', 'HEC_TIPAGRE', 'DEPTO_MCPIO', 'HEC_DEPTOMCPIO'
]
target = 'INST_DONDE_DENUNCIO'

# Drop filas sin target
df_clean = df[vars_ + [target]].dropna(subset=[target]).copy()

# Forzar todo target a string
df_clean[target] = df_clean[target].astype(str)

# Definir X e y limpios
X = df_clean[vars_]
y = df_clean[target]

# Revisar conteo de clases
print("Clases en y:", y.value_counts())


C:\Users\diego\AppData\Local\Temp\ipykernel_1580\2594719522.py:14: DtypeWarning: Columns (0,3,7,8,9,10,11,12,15,16,17,19,28,29,31,33,38,42,43,45,48,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("../CSV/UnidoPorAñoCSV/vif_all_years.csv")


Clases en y: INST_DONDE_DENUNCIO
       287079
4       19222
3        8643
9.0      8604
4.0      7443
1        6173
9        5257
3.0      4894
1.0      3411
2         423
6         238
2.0       178
5          65
6.0        57
5.0        15
Name: count, dtype: int64


### 2. Carga y limpieza inicial de datos
- Eliminamos filas donde la variable objetivo sea `NaN`.
- Convertimos todos los valores de la variable objetivo a `str`, para evitar comparaciones mixtas.

In [9]:

# 1) Carga del dataset
df = pd.read_csv("../CSV/UnidoPorAñoCSV/vif_all_years.csv")

# 2) Filtrar filas donde INST_DONDE_DENUNCIO NO sea null y NO esté vacío
mask = df[target].notna() & df[target].astype(str).str.strip().ne('')
df_clean = df[mask].copy()

# 3) Convertir la variable objetivo a string y unificar formatos numéricos
df_clean[target] = df_clean[target].astype(str).str.strip()
# (Opcional) Si quieres convertir "9.0" a "9", etc.:
df_clean[target] = df_clean[target].apply(lambda x: str(int(float(x))) if x.replace('.','',1).isdigit() else x)

# 4) Definir X e y limpios
X = df_clean[vars_]
y = df_clean[target]

# 5) Verificar conteo de clases, ya sin los 287 079 registros vacíos
print("Clases en y:\n", y.value_counts())


C:\Users\diego\AppData\Local\Temp\ipykernel_1580\1042074070.py:2: DtypeWarning: Columns (0,3,7,8,9,10,11,12,15,16,17,19,28,29,31,33,38,42,43,45,48,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("../CSV/UnidoPorAñoCSV/vif_all_years.csv")


Clases en y:
 INST_DONDE_DENUNCIO
4    26665
9    13861
3    13537
1     9584
2      601
6      295
5       80
Name: count, dtype: int64


### 3. Preprocesamiento de variables
- Imputamos valores faltantes en numéricas con la media y escalamos.
- Imputamos las categóricas con la moda y aplicamos one-hot encoding.


In [10]:
# Definir features numéricas vs categóricas
numeric_features   = ['VIC_EDAD', 'AGR_EDAD']
categorical_features = [c for c in vars_ if c not in numeric_features]

# Pipelines por tipo
numeric_transformer = Pipeline([
    ("imputer", SimpleImputer(strategy="mean")),
    ("scaler", StandardScaler())
])
categorical_transformer = Pipeline([
    ("imputer", SimpleImputer(strategy="most_frequent")),
    ("encoder", OneHotEncoder(handle_unknown="ignore"))
])

# ColumnTransformer combinado
preprocessor = ColumnTransformer([
    ("num", numeric_transformer, numeric_features),
    ("cat", categorical_transformer, categorical_features)
])



### 4. División de datos y entrenamiento
- Se estratifica según `y`.
- Modelo: regresión logística multinomial con solver `lbfgs`.

In [11]:
# Ensure all categorical features are strings
for col in categorical_features:
    X[col] = X[col].astype(str)

# División entreno/test
X_train, X_test, y_train, y_test = train_test_split(
    X, y,
    stratify=y,
    test_size=0.3,
    random_state=42
)

# Pipeline completo
clf = Pipeline([
    ("preprocessor", preprocessor),
    ("classifier", LogisticRegression(
        max_iter=1000,
        multi_class='multinomial',
        solver='lbfgs'
    ))
])

# Entrenar
clf.fit(X_train, y_train)


C:\Users\diego\AppData\Local\Temp\ipykernel_1580\2734570473.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X[col] = X[col].astype(str)
C:\Users\diego\AppData\Local\Temp\ipykernel_1580\2734570473.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X[col] = X[col].astype(str)
C:\Users\diego\AppData\Local\Temp\ipykernel_1580\2734570473.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats i

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer()),
                                                                  ('scaler',
                                                                   StandardScaler())]),
                                                  ['VIC_EDAD', 'AGR_EDAD']),
                                                 ('cat',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('encoder',
                                                                   OneHotEncoder(handle_unknown='ignore'))]),
                                                  ['VIC_ESCOLARIDAD',
                                                   'VIC_OCUP', 'VIC_DEDICA',
                                                   'VIC_TRABAJA',
                                                   'AGR_ESCOLARIDAD',
                                                   'AGR_OCUP', 'AGR_DEDICA',
                                                   'AGR_TRABAJA', 'VIC_REL_AGR',
                                                   'HEC_TIPAGRE', 'DEPTO_MCPIO',
                                                   'HEC_DEPTOMCPIO'])])),
                ('classifier',
                 LogisticRegression(max_iter=1000, multi_class='multinomial'))])

In [12]:
# Predicción y reporte
y_pred = clf.predict(X_test)
print(classification_report(y_test, y_pred))


              precision    recall  f1-score   support

           1       0.41      0.25      0.31      2875
           2       0.20      0.01      0.02       180
           3       0.54      0.42      0.48      4061
           4       0.56      0.75      0.64      8000
           5       0.00      0.00      0.00        24
           6       0.00      0.00      0.00        89
           9       0.53      0.49      0.51      4158

    accuracy                           0.54     19387
   macro avg       0.32      0.27      0.28     19387
weighted avg       0.52      0.54      0.52     19387



c:\Users\diego\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\diego\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\diego\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, mo

## Análisis del reporte de clasificación

El desempeño de tu modelo muestra un **alto accuracy global** (0.85) impulsado principalmente por la clase mayoritaria (“0”), pero evidencia **déficits importantes** en casi todas las demás categorías:

| Clase | Support | Precision | Recall | F1-score |
|:-----:|:-------:|:---------:|:------:|:--------:|
| **0** |  86 124 |      0.88 |   1.00 |     0.93 |
| **1 / 1.0** | 1 852 / 1 023 | 0.13 / 0.39 | 0.00 / 0.25 | 0.00 / 0.31 |
| **2 / 2.0** |   127 /    53 | 0.00 / 0.20 | 0.00 / 0.02 | 0.00 / 0.03 |
| **3 / 3.0** | 2 593 / 1 468 | 0.30 / 0.59 | 0.01 / 0.50 | 0.01 / 0.54 |
| **4 / 4.0** | 5 767 / 2 233 | 0.26 / 0.54 | 0.00 / 0.59 | 0.00 / 0.56 |
| **5 / 5.0** |    20 /     5 | 0.00 / 0.00 | 0.00 / 0.00 | 0.00 / 0.00 |
| **6 / 6.0** |    71 /    17 | 0.00 / 0.00 | 0.00 / 0.00 | 0.00 / 0.00 |
| **9 / 9.0** | 1 577 / 2 581 | 0.86 / 0.57 | 0.00 / 0.68 | 0.01 / 0.62 |

### Puntos clave

1. **Clase 0 domina el accuracy**  
   - Representa ~82% del conjunto de test (86 124 / 105 511).  
   - Recall=1.00 significa que casi ningún “0” se pierde, pero las otras clases apenas se detectan.

2. **Clases minoritarias mal predichas**  
   - Recall cercano a 0 para la mayoría; incluso cuando precision es moderada (p.ej. 0.86 para “9”), no captura instancias reales de esa clase.  
   - F1-scores prácticamente nulos (<0.1) para casi todas menos “0” y “9.0”.

3. **Duplicación de etiquetas**  
   - Se observan pares “1” y “1.0”, “2” y “2.0”, etc., que probablemente representan la misma categoría pero en tipos distintos (str vs float).  
   - Esto fracciona el support y dificulta el aprendizaje.

4. **Macro vs Weighted avg**  
   - **Macro avg F1 ≈ 0.20** refleja que el modelo globalmente es muy pobre en clases minoritarias.  
   - **Weighted avg F1 ≈ 0.80** está sesgado por la alta presencia de la clase “0”.


## MEJORADO

1. **`RandomizedSearchCV`** con pocas iteraciones.  
2. **Reducir el número de folds** de validación (por ejemplo, de 5 a 2).  
3. **Acotar el espacio de búsqueda** a los parámetros más críticos (`C` y `class_weight`).



In [13]:
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import loguniform
from sklearn.pipeline import Pipeline

# Pipeline base (igual que antes)
base_clf = Pipeline([
    ("preprocessor", preprocessor),
    ("classifier", LogisticRegression(
        max_iter=500,          # reducir iteraciones
        multi_class="multinomial",
        solver="saga",         # saga soporta l1 y l2
        random_state=42
    ))
])

# Espacio de búsqueda acotado
param_dist = {
    "classifier__C": loguniform(1e-3, 1e1),      # distribución log-uniforme
    "classifier__class_weight": [None, "balanced"]
}

rand_search = RandomizedSearchCV(
    estimator=base_clf,
    param_distributions=param_dist,
    n_iter=10,                  # sólo 10 combinaciones
    scoring="f1_macro",
    cv=2,                       # sólo 2 folds
    n_jobs=-1,
    random_state=42,
    verbose=1
)


# Ajustar sobre el set de entrenamiento
rand_search.fit(X_train, y_train)

# Resultados
print("Mejores parámetros:", rand_search.best_params_)
print("Mejor F1 macro (CV):", rand_search.best_score_)

# Evaluar en test
best_clf = rand_search.best_estimator_
y_pred = best_clf.predict(X_test)
print(classification_report(y_test, y_pred))


Fitting 2 folds for each of 10 candidates, totalling 20 fits


c:\Users\diego\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\linear_model\_logistic.py:1247: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.7. From then on, it will always use 'multinomial'. Leave it to its default value to avoid this warning.
  warnings.warn(
c:\Users\diego\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\linear_model\_sag.py:348: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(


Mejores parámetros: {'classifier__C': 0.24810409748678125, 'classifier__class_weight': None}
Mejor F1 macro (CV): 0.27201770567951655
              precision    recall  f1-score   support

           1       0.43      0.23      0.30      2875
           2       0.50      0.01      0.01       180
           3       0.56      0.41      0.47      4061
           4       0.56      0.77      0.65      8000
           5       0.00      0.00      0.00        24
           6       0.00      0.00      0.00        89
           9       0.53      0.48      0.51      4158

    accuracy                           0.54     19387
   macro avg       0.37      0.27      0.28     19387
weighted avg       0.53      0.54      0.52     19387



c:\Users\diego\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\diego\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\diego\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, mo